In [3]:
#install tweepy and kafka-python as requirements
#!pip install tweepy kafka-python

In [4]:
import tweepy
import datetime
import pandas as pd
from kafka import KafkaProducer

#BearerToken is from Twitter-API-Account and is intentionally left blank in the repository
bearerToken = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

#The kafka server is running in the same docker-compose, therefore it is accessible via the hostname kafka
kafka_server = "kafka:9092"

#We used tweets as the name of the kafka topic
topic_name = "tweets"

producer = KafkaProducer(bootstrap_servers=kafka_server)

cl = tweepy.Client(bearer_token=bearerToken)

#create datetime 7 days ago
start = datetime.datetime.now() - datetime.timedelta(days=7)
hashtag = "#tsla"

#get tweet count for hashtag
res = cl.get_recent_tweets_count(hashtag,start_time=start)


In [5]:
#convert data to dataframe
df = pd.DataFrame.from_dict(res.data)

#skip first and last entry because they are no full hours
df = df.iloc[1:-1]

#convert types
df['tweet_count'].astype(int)
df['time'] = pd.to_datetime(df['start'])
#remove start and end
df = df.drop(['start','end'],axis=1)

In [6]:
#We can take a look at the data from the Twitter API
df

,tweet_count,time
1,52,2023-01-14 16:00:00+00:00
2,44,2023-01-14 17:00:00+00:00
3,52,2023-01-14 18:00:00+00:00
4,17,2023-01-14 19:00:00+00:00
5,27,2023-01-14 20:00:00+00:00
...,...,...
163,20,2023-01-21 10:00:00+00:00
164,11,2023-01-21 11:00:00+00:00
165,22,2023-01-21 12:00:00+00:00
166,29,2023-01-21 13:00:00+00:00


In [14]:
#write pandas timeseries to kafka topic
for index, row in df.iterrows():
    data = row.to_json().encode('utf-8')
    producer.send(topic_name, data)

# Flush the producer to ensure all data is sent
producer.flush()